In [2]:
#Script to load trained model and perform translations

import itertools

import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer

EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256


CUDA_DEVICE = 0

#Loading the reader, vocab, embeddings and model structure 
reader = Seq2SeqDatasetReader(
    source_tokenizer=WordTokenizer(),
    target_tokenizer=CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy = True)

vocab = Vocabulary.from_files("/home/infili/translation/Translation/trained/20200119/vocabulary_films_ana7_20200119/")


en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=EN_EMBEDDING_DIM)

encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

attention = DotProductAttention()

max_decoding_steps = 300  
model_pred = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)

# Reload the trained model.
with open("/home/infili/translation/Translation/trained/20200119/model_greek_films_ana7_20200119.th", 'rb') as f:
    model_pred.load_state_dict(torch.load(f))
    model_pred.eval()


In [67]:
#Predict on new text using loaded model

predictor = SimpleSeq2SeqPredictor(model_pred, dataset_reader=reader)


#To translate, write comma-separated sentences in quotes:

test = [
    "Καλημέρα!",
    "Πόσο χρόνο έχουμε μέχρι το επόμενο διάλειμμα;",
    "Αυτή είναι μια καλή προσπάθεια για μετάφραση",
    "Ίσως θα μπορούσε να είναι καλύτερη.",
    "Αλλά με τόσο λίγα δεδομένα, δεν περίμενα και πολλά.",
    "Θα δω αν μπορώ να αυξήσω το σύνολο εκπαίδευσης",
    "Εντωμεταξύ, ας δοκιμάσουμε μερικά παραδείγματα",
    "Άλλωστε αυτό κάνουμε τόση ώρα, σωστά;",
    "Ας ξεκινήσουμε με τα βασικά:",
    "Ναι, αυτό θα μπορούσαμε να το κάνουμε",
    "Πως σε λένε;",
    "Μπορώ να κάνω κάτι για σένα;",
    "Και τώρα κάτι πιο δύσκολο:",
    "Μου την έφερες!",
    "Νιώθω πως κάτι περίεργο συμβαίνει.",
    "Πεινάω από τη στιγμή που ξύπνησα",
    "Θα βγω έξω μια βολτα στο δάσος",
    "Επιστρέφω σε 10 λεπτά.",
    "Γύρισα από το σχολείο κι έπεσα για ύπνο αμέσως.",
    "Θα πάμε διακοπές στο Παρίσι!",
    "Είμαι λίγο αγχωμένος για τις εξετάσεις...",
    "Φυσικά και ο φίλος σου μπορεί να έρθει!",
    "Όταν γράφω μεταφορικά, είναι δύσκολο να μεταφράσω σωστά.",
    "Είναι ξεκάθαρο ότι μερικές προτάσεις δε βγάζουν νόημα",
    "Πάμε και σε μερικές κριτικές:",
    "Ο χώρος και το φαγητό ήταν τέλεια!",
    "Μου έριξε χυλόπιτα",
    "Τρελαίνομαι για πουρέ.",
    "Όλα απαίσια, μην κάνετε τον κόπο",
    "Δεν με ενθουσίασε ιδιαίτερα αλλά θα του έδινα μια δεύτερη ευκαιρία.",
    "Η Κατερίνα λέει ότι είσαι τέλειο!",
    "Σε έχω ρωτήσει τόσες φορές αν τελικά θα βγούμε και δεν μου έχεις απαντήσει ακόμα.",
    "Ο Γιώργος έχασε το πορτοφόλι του, πήγε στην αστυνομία και το βρήκαν.",
    "Αρκετά για σήμερα, τα λέμε αύριο",
    "Είναι μια τέλεια μέρα, είμαστε έτοιμοι για νέες περιπέτειες!",
    "Δώσε μου τον κώδικα και θα τον κοιτάξω όταν βρω χρόνο.",
    "Δε θα βάλω οινόπνευμα, τσούζει!",
    "Σταμάτα να αμφισβητείς τις ικανότητές μου",
    "Το φαγητό ήταν πεντανόστιμο και οι τιμές μια χαρουλα.Παρα πολύ τίμιο.",
    "Είμαι βαθύτατα συγκινημένος.",
    "Το παιδί είναι καθυστερημένο."
]

#Parsing the output to remove quotes and irrelevant characters

import re
regex = r"', '"
regex2 = r", \"'\", '"
subst = ""
subst2 = ""

for i in test:
    p = predictor.predict(i)['predicted_tokens']
    result = re.sub(regex, subst, str(p), 0, re.MULTILINE | re.IGNORECASE)
    result = re.sub(regex2, subst2, str(result), 0, re.MULTILINE | re.IGNORECASE)

    print(i,result) 

Καλημέρα! ['good morning!']
Πόσο χρόνο έχουμε μέχρι το επόμενο διάλειμμα; ['how long have we have the next break?']
Αυτή είναι μια καλή προσπάθεια για μετάφραση ['this is a good try to translate.']
Ίσως θα μπορούσε να είναι καλύτερη. ['maybe he could be better.']
Αλλά με τόσο λίγα δεδομένα, δεν περίμενα και πολλά. ['but with so few data, i didn't expect a lot.']
Θα δω αν μπορώ να αυξήσω το σύνολο εκπαίδευσης ['i'll see if i can raise the training.']
Εντωμεταξύ, ας δοκιμάσουμε μερικά παραδείγματα ['meanwhile, let's try some examples.']
Άλλωστε αυτό κάνουμε τόση ώρα, σωστά; ['besides, what about this, right?']
Ας ξεκινήσουμε με τα βασικά: ['let's start with the basic,']
Ναι, αυτό θα μπορούσαμε να το κάνουμε ['yeah, that we could do it.']
Πως σε λένε; ['what's your name?']
Μπορώ να κάνω κάτι για σένα; ['can i do something for you?']
Και τώρα κάτι πιο δύσκολο: ['and now something hard:']
Μου την έφερες! ['you brought me!']
Νιώθω πως κάτι περίεργο συμβαίνει. ['i feel something weird.']
Πειν